In [ ]:
from concurrent.futures import ProcessPoolExecutor

from timecraft_ai import DatabaseConnector
from timecraft_ai import TimeCraftModel

**Define the function to process a product**

In [ ]:
def process_product(product_id):
    # Set up the database connector
    db_connector = DatabaseConnector(
        db_type="mssql",
        username="sankhya",
        password="azsxdc",
        dsn="127.0.0.1:1433/sankhya_prod"
    )

**Define the SQL query to extract historical stock data**

In [ ]:
    with open("data/EST_X_PROD_X_DATE-MSSQL.sql.j2", "r") as file:
        query_template = file.read()

**Render the template with the product ID**

In [ ]:
    query = query_template.format({"product_id": product_id})

    ts_model = TimeCraftModel(
        db_connector=db_connector,
        query=query.replace("{ product_id }", str(product_id)),
        date_column="DTNEG",
        value_columns=["SALDO_HISTORICO"],
        is_csv=False,
        periods=30
    )

    try:

**Run the model and generate forecasts**

In [ ]:
ts_model.run()
        # forecast = ts_model.get_forecast()
    except Exception as e:
        print(f"Error processing product {product_id}: {e}")
        return None
    
    print(f"Forecast for product {product_id} completed.")
    # print(forecast)

**Save forecasts**

In [ ]:
    output_file = f"output/products_stock/forecast_estoque_{product_id}.csv"
    ts_model.save_forecast(output_file)

**Plot forecasts**

In [ ]:
    # ts_model.plot_forecast()
    ts_model.save_plots(f"output/forecast_estoque_{product_id}.png")

**Define the function to get product IDs**

In [ ]:
def get_product_ids() -> list:
    query_products = """
    SELECT 
        P.CODPROD
    FROM 
        TGFPRO P
    WHERE 
        P.ATIVO = 'S'
        AND P.CODPROD IN(7943, 12614, 5919, 12412, 127977)
    """

    

** **Conectar ao banco de dados**

In [ ]:
    db_connector = DatabaseConnector(
        db_type="mssql",
        username="sankhya",
        password="azsxdc",
        dsn="127.0.0.1:1433/sankhya_prod"
    )
    db_connector.connect()

    try:

**Query the product list**

In [ ]:
        products_df = db_connector.execute_query(query_products)
    except Exception as e:
        print(f"Erro ao consultar produtos: {e}")
        return []

    # Fechar a conexão
    db_connector.close()

    # Lista de IDs dos produtos
    product_ids = products_df["CODPROD"].tolist()

    return product_ids

**Get product IDs**

In [ ]:
product_ids = get_product_ids()

**Process products in parallel**

In [ ]:
with ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(process_product, product_ids)

print("Processamento concluído.")

**End of script**